In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

In [17]:
from pydub import AudioSegment
import soundfile as sf
from params import EXCERPT_LENGTH,INPUT_DIR_PARENT,OUTPUT_DIR
# sys.path.insert(0, './models/audioset')

# from vggish_params import EXAMPLE_HOP_SECONDS
from pre_process_func import cal_sample_size,iterate_for_waveform_to_examples
import os 
import math

from pathlib import Path

In [13]:
import numpy as np
import sys
sys.path.insert(0, './models/audioset')
from models.audioset import vggish_slim
from models.audioset import vggish_params
from models.audioset import vggish_input
from models.audioset import vggish_postprocess




/scratch/enis/conda/envs/speechEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/scratch/enis/conda/envs/speechEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/scratch/enis/conda/envs/speechEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/scratch/enis/conda/envs/speechEnv/lib/python3


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [3]:
src="/tank/data/nna/real/11A/"
file="/tank/data/nna/real/11A/S4A10276_20190514_153000.flac"
# flac_file = AudioSegment.from_file(file, "flac")
# ffmpeg -i /tank/data/nna/real/11A/S4A10276_20190514_153000.flac -map 0 -c copy -f segment -segment_time "02:00:00" ./output_%03d.flac


In [18]:
flacs=os.listdir("/tank/data/nna/real/11A")
wav_data, sr = sf.read(Path(src+flac[0]),dtype='int16')



In [9]:
import soundfile as sf
wav_data, sr = sf.read(Path("/scratch/enis/data/nna/real/39B/S4A10262_20190611_101602_segments/output000.flac"),dtype='int16')


In [10]:
sf.__version__

'0.10.2'

In [15]:
for flac in flacs[1:2]:
    wav_data, sr = sf.read("/scratch/enis/data/nna/real/27A/S4A10251_20190504_000000_segments/output000.flac",dtype='int16')

    sample_size,offset,remainder_wav_data,lower_limit=cal_sample_size(wav_data,sr)
    print(sample_size,offset,remainder_wav_data,lower_limit)
    
    
    sound=iterate_for_waveform_to_examples2(wav_data,sr)
    print(sound.shape)

4438 480000 384000 138960.0
213024000


KeyboardInterrupt: 

In [7]:
len(wav_data)/offset

443.8

In [10]:
def iterate_for_waveform_to_examples2(wav_data,sr):
    """Wrapper for waveform_to_examples from models/audioset/vggish_input.py

        Iterate over data with 10 seconds batches, so waveform_to_examples produces
        stable results (equal size)
        read **(16/06/2019)** at Project_logs.md for explanations.

        Args:
            wav_data (numpy.array): audio data in wav format
            sr (int): sampling rate of the audio

        Returns:
            See waveform_to_examples.
    """
    sample_size,offset,remainder_wav_data,lower_limit=cal_sample_size(wav_data,sr)
    # in this loop wav_data jumps offset elements and sound jumps EXCERPT_LENGTH*2
    # because offset number of raw data turns into EXCERPT_LENGTH*2 pre-processed
    sound=np.zeros((sample_size,96,64),dtype=np.float32)
    count=0
    print(len(wav_data))
    for i in range(0,len(wav_data),offset):
    #this is when wav_data%offset!=0
        # numpy indexing handles bigger indexes
        # i+offset>len(wav_data) means that we are on the last loop
        # then if there is enough remaind data, process it otherwise not
        if i+offset>len(wav_data) and remainder_wav_data<lower_limit:
            continue
        # left data is smaller than 22712, we cannot pre-process
        # if smaller than 42998, will be 0 anyway
        a_sound= vggish_input.waveform_to_examples(wav_data[i:i+(offset)], sr)
        sound[count:(count+a_sound.shape[0]),:,:]=a_sound[:,:,:]
        count+=a_sound.shape[0]
    return sound

In [62]:

print(len(data),data.shape,sound.shape)

213024000 (213024000, 2) (4439, 96, 64)


In [100]:
# vggfile="/scratch/enis/data/nna/NUI_DATA/01 Itkillik/August 2016/ITKILLIK1_20160727_135107_vgg/ITKILLIK1_20160727_135107_rawembeddings049.npy"

# vggnp=np.load(vggfile)

In [99]:
# vggfile="/scratch/enis/data/nna/NUI_DATA/01 Itkillik/August 2016/ITKILLIK1_20160727_135107_vgg/ITKILLIK1_20160727_135107_rawembeddings048.npy"

# vggnp8=np.load(vggfile)

In [98]:
# vggnp[-1]

In [97]:
# vggnp8[-1]

In [96]:
# vggfile="/scratch/enis/data/nna/NUI_DATA/01 Itkillik/August 2016/ITKILLIK1_20160731_171332_vgg/ITKILLIK1_20160731_171332_rawembeddings048.npy"
# vggnp8_2=np.load(vggfile)

In [95]:
# vggnp8_2[-1]

In [94]:
# name="ITKILLIK1_20160825_132756"
# vggfile="/scratch/enis/data/nna/NUI_DATA/01 Itkillik/August 2016/{}_vgg/{}_rawembeddings049.npy".format(name,name)
# vggnp9_2=np.load(vggfile)
# vggnp9_2[-1]

In [93]:
# vggnp9_2[-1]

In [144]:
wav_data=data[:]
sr=samplerate
vggish_params.EXAMPLE_HOP_SECONDS=0.96

initial=-1
# 22712 0 starts ?? 22712
# 42998 1 starts ?? 42998
# 63284 2 starys ?? 85334
        # 3        #  ?? 127734
    # 63284-42998 = 20286 !    ?? 42336
    #               297014
#       8  starts  339350
#       9 starts   381686    
#       10 starts  424022   
#       11 starts  466358 
# 22712+(seconds*20286)=len_wav_data

for index in range(1,11):
    for i in range(int((sr*0.96*index)+(719)),(EXCERPT_LENGTH+1)*sr,1):
        #   print(i)
        a_sound= vggish_input.waveform_to_examples(wav_data[0:i], sr)
        #   print(i," ",a_sound.shape)
        if a_sound.shape[0]==index:
            print("shape is",a_sound.shape[0],"starting from ",i)

            initial=a_sound.shape[0]
            break
        if i%1==0:
            print(i)
#   break
# for 0.96
# shape is 1 starting from  42998
# shape is 2 starting from  85334
# shape is 3 starting from  127670
# shape is 4 starting from  170006
# shape is 5 starting from  212342
# shape is 6 starting from  254678
# shape is 7 starting from  297014
# shape is 8 starting from  339350
# shape is 9 starting from  381686
# shape is 10 starting from  424022
# shape is 11 starting from  466358

# for flac with 48000 sampling rate
# shape is 1 starting from  46800
# shape is 2 starting from  92880


46799
shape is 1 starting from  46800
92879
shape is 2 starting from  92880
138959
shape is 3 starting from  138960
185039
shape is 4 starting from  185040
231119
shape is 5 starting from  231120
277199
shape is 6 starting from  277200
323279
shape is 7 starting from  323280
369359
shape is 8 starting from  369360
415439
shape is 9 starting from  415440
461519
shape is 10 starting from  461520


In [59]:
import time
import resampy

s=time.time()
if len(data.shape) > 1:
    datas1 = np.mean(data[:213024000], axis=1)
datas = resampy.resample(datas1, 41000, 16000)


e=time.time()
t1=(e-s)

s=time.time()
if len(data.shape) > 1:
    datas1 = np.mean(data[:4100000], axis=1)
datas2 = resampy.resample(datas1, 41000, 16000)


e=time.time()
t2=(e-s)
print((213024000/4100000)*t2/t1)

4.148260593414307


In [5]:
from params import LOGS_FILE
import pre_process_func
from pathlib import Path
from subprocess import Popen, PIPE

import numpy as np

TEST_DIR=Path("tests/")
EXAMPLE_MODELS_DIR=TEST_DIR / "example_models/"
EXAMPLE_OUTPUT_DIR= TEST_DIR / "example_outputs"
OUTPUT_DIR=Path("aggregates")

root_dir="/tank/data/nna/real/"
# input_path_list=[root_dir+"tests/data/3hours30min.mp3",]

input_path_list=[root_dir+"39B/S4A10262_20190611_101602.flac",]

pre_process_func.parallel_pre_process(input_path_list,
                                        output_dir="./tests/data/output",
                                        input_dir_parent=root_dir,
                                        cpu_count=2,
                                        segment_len="02:00:00",
                                        logs_file_path=LOGS_FILE)